In [5]:
import pandas as pd

# Giriş ve çıkış dosyalarının yolları
input_file = "data_file/data.csv"  # Orijinal CSV dosyası
output_file = "first_10k.csv"  # Yeni oluşturulacak CSV dosyası

# Dosyayı yükleyip ilk 10k satırı alma
data = pd.read_csv(input_file)
first_10k = data.head(10000)

# Yeni dosyaya kaydetme
first_10k.to_csv(output_file, index=False)

print(f"İlk 10.000 satır {output_file} dosyasına kaydedildi.")


İlk 10.000 satır first_10k.csv dosyasına kaydedildi.


In [ ]:
import csv

# Veri dosyasının adı
input_file = "data_file/output_positions.csv"

# Veriyi oku ve işlemek için bir listeye aktar
data = []
with open(input_file, "r") as file:
    reader = csv.reader(file)
    headers = next(reader)  # Başlık satırını atla
    for row in reader:
        # Epoch hariç diğer sütunları float olarak işle
        data.append([float(value) for value in row[:-1]])

# Her sütun için ortalama, maksimum ve minimum değerleri hesapla
column_stats = {}
for col_idx in range(len(data[0])):
    column_values = [row[col_idx] for row in data]
    col_mean = sum(column_values) / len(column_values)
    col_max = max(column_values)
    col_min = min(column_values)
    max_row_index = column_values.index(col_max) + 1  # Satır numarası (1 tabanlı)
    min_row_index = column_values.index(col_min) + 1  # Satır numarası (1 tabanlı)
    column_stats[col_idx + 1] = {
        "mean": col_mean,
        "max": col_max,
        "max_row": max_row_index,
        "min": col_min,
        "min_row": min_row_index,
    }

# Sonuçları ekrana yazdır
for col_idx, stats in column_stats.items():
    print(
        f"{col_idx}. column ortalama: {stats['mean']:.6f}, "
        f"max: {stats['max']:.6f} (satır {stats['max_row']}), "
        f"min: {stats['min']:.6f} (satır {stats['min_row']})"
    )


1. column ortalama: 173.680927, max: 359.933190 (satır 42), min: 0.011030 (satır 2597)
2. column ortalama: 179.969303, max: 359.997050 (satır 3753), min: 0.062640 (satır 2146)
3. column ortalama: 170.353747, max: 359.945300 (satır 4504), min: 0.037640 (satır 247)
4. column ortalama: 7.819930, max: 64.004780 (satır 2080), min: 0.048370 (satır 7395)
5. column ortalama: 0.146493, max: 0.889806 (satır 3124), min: 0.001390 (satır 9812)
6. column ortalama: 0.235961, max: 1.298651 (satır 2078), min: 0.007900 (satır 9220)
7. column ortalama: 2.698952, max: 24.968533 (satır 9220), min: 0.832034 (satır 2078)


In [7]:
import math
import csv
from datetime import datetime

# CSV dosyasını oku ve yeni dosyaya yaz
input_file = "first_10k.csv"
output_file = "10k_epoch.csv"

# Kepler Denklemini çözmek için Newton-Raphson yöntemi
def solve_kepler(M, e, tolerance=1e-6):
    E = M  # İlk tahmin
    iteration = 0
    while True:
        E_next = E - (E - e * math.sin(E) - M) / (1 - e * math.cos(E))
        if abs(E_next - E) < tolerance:
            break
        E = E_next
        iteration += 1
    return E


# Fonksiyon: Gerçek anomalinin ve diğer parametrelerin hesaplanması
def calculate_position(row):
    # Yörünge elemanlarını al
    M = math.radians(float(row[0]))        # Ortalama Anomali
    omega = math.radians(float(row[1]))  # Periapsis boylamı
    Omega = math.radians(float(row[2]))  # Yükselen düğüm boylamı
    I = math.radians(float(row[3]))  # Eğiklik
    e = float(row[4])         # Eksantriklik
    a = float(row[6])         # Yarı büyük eksen (AU)


    # Eksen anomali E hesaplanır
    E_rad = solve_kepler(M, e)

    # Gerçek anomali ν hesaplanır
    nu_rad = 2 * math.atan2(
        math.sqrt(1 + e) * math.sin(E_rad / 2),
        math.sqrt(1 - e) * math.cos(E_rad / 2)
    )

    # Yarıçap vektörü r hesaplanır
    r = a * (1 - e * math.cos(E_rad))

    # Perifokal koordinatlar (x', y')
    x_prime = r * math.cos(nu_rad)
    y_prime = r * math.sin(nu_rad)

    # Dönüşüm için gerekli trigonometrik değerler
    cos_Omega = math.cos(Omega)
    sin_Omega = math.sin(Omega)
    cos_I = math.cos(I)
    sin_I = math.sin(I)
    cos_omega = math.cos(omega)
    sin_omega = math.sin(omega)

    # Ekliptik koordinatlara dönüşüm
    x = (cos_Omega * cos_omega - sin_Omega * sin_omega * cos_I) * x_prime + \
        (-cos_Omega * sin_omega - sin_Omega * cos_omega * cos_I) * y_prime

    y = (sin_Omega * cos_omega + cos_Omega * sin_omega * cos_I) * x_prime + \
        (-sin_Omega * sin_omega + cos_Omega * cos_omega * cos_I) * y_prime

    z = (sin_omega * sin_I) * x_prime + (cos_omega * sin_I) * y_prime

    return x, y, z

with open(input_file, "r") as infile, open(output_file, "w", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Başlıkları yaz
    headers = next(reader)  # İlk satır başlıklar
    writer.writerow(headers + ["x", "y", "z"])

    # Her satır için hesaplamaları yap ve yaz
    for row in reader:
        x, y, z = calculate_position(row)
        writer.writerow(row + [f"{x:.7f}", f"{y:.7f}", f"{z:.7f}"])

print(f"Konum verileri {output_file} dosyasına kaydedildi.")


Konum verileri 10k_epoch.csv dosyasına kaydedildi.


In [9]:
import math
import csv
from datetime import datetime

entered_date = "20260101"

# CSV dosyasını oku ve yeni dosyaya yaz
input_file = "first_10k.csv"
output_file = "10k_20260101.csv"

entered_date_obj = datetime.strptime(entered_date, "%Y%m%d")
entered_epoch_time = int(entered_date_obj.timestamp())

# Kepler Denklemini çözmek için Newton-Raphson yöntemi
def solve_kepler(M, e, tolerance=1e-6):
    E = M  # İlk tahmin
    iteration = 0
    while True:
        E_next = E - (E - e * math.sin(E) - M) / (1 - e * math.cos(E))
        if abs(E_next - E) < tolerance:
            break
        E = E_next
        iteration += 1
    return E


# Fonksiyon: Gerçek anomalinin ve diğer parametrelerin hesaplanması
def calculate_position(row):
    # Yörünge elemanlarını al
    M0 = math.radians(float(row[0]))        # Ortalama Anomali
    omega = math.radians(float(row[1]))  # Periapsis boylamı
    Omega = math.radians(float(row[2]))  # Yükselen düğüm boylamı
    I = math.radians(float(row[3]))  # Eğiklik
    e = float(row[4])         # Eksantriklik
    n = float(row[5])         # Günlük ortalama hareket (radyan/gün)
    a = float(row[6])         # Yarı büyük eksen (AU)
    epoch = datetime.strptime(row[7], "%Y%m%d")
    epoch_time = int(epoch.timestamp())

    # Gün farkını hesaplama
    difference_in_seconds = abs(entered_epoch_time - epoch_time)
    difference_in_days = difference_in_seconds // (24 * 3600)

    # Ortalama anomaliyi güncelleme ve mod 2*pi uygulama
    M = (M0 + math.radians(n * difference_in_days)) % (2 * math.pi)

    
    # Eksen anomali E hesaplanır
    E_rad = solve_kepler(M, e)

    # Gerçek anomali ν hesaplanır
    nu_rad = 2 * math.atan2(
        math.sqrt(1 + e) * math.sin(E_rad / 2),
        math.sqrt(1 - e) * math.cos(E_rad / 2)
    )

    # Yarıçap vektörü r hesaplanır
    r = a * (1 - e * math.cos(E_rad))

    # Perifokal koordinatlar (x', y')
    x_prime = r * math.cos(nu_rad)
    y_prime = r * math.sin(nu_rad)

    # Dönüşüm için gerekli trigonometrik değerler
    cos_Omega = math.cos(Omega)
    sin_Omega = math.sin(Omega)
    cos_I = math.cos(I)
    sin_I = math.sin(I)
    cos_omega = math.cos(omega)
    sin_omega = math.sin(omega)

    # Ekliptik koordinatlara dönüşüm
    x = (cos_Omega * cos_omega - sin_Omega * sin_omega * cos_I) * x_prime + \
        (-cos_Omega * sin_omega - sin_Omega * cos_omega * cos_I) * y_prime

    y = (sin_Omega * cos_omega + cos_Omega * sin_omega * cos_I) * x_prime + \
        (-sin_Omega * sin_omega + cos_Omega * cos_omega * cos_I) * y_prime

    z = (sin_omega * sin_I) * x_prime + (cos_omega * sin_I) * y_prime

    return M, x, y, z


with open(input_file, "r") as infile, open(output_file, "w", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Başlıkları yaz
    headers = next(reader)  # İlk satır başlıklar
    writer.writerow(["updated_M"] + headers[1:-1] + ["updated_date", "x", "y", "z"])

    # Her satır için hesaplamaları yap ve yaz
    for row in reader:
        M, x, y, z = calculate_position(row)
        writer.writerow([f"{math.degrees(M):.5f}"] + row[1:-1] + [entered_date] + [f"{x:.7f}", f"{y:.7f}", f"{z:.7f}"])


print(f"Konum verileri {output_file} dosyasına kaydedildi.")


Konum verileri 10k_20260101.csv dosyasına kaydedildi.


In [10]:
import math
import csv
from datetime import datetime

entered_date = "20260101"

# CSV dosyasını oku ve yeni dosyaya yaz
input_file = "first_10k.csv"
output_file = "10k_20260101.csv"

entered_date_obj = datetime.strptime(entered_date, "%Y%m%d")
entered_epoch_time = int(entered_date_obj.timestamp())

# Kepler Denklemini çözmek için Newton-Raphson yöntemi
def solve_kepler(M, e, tolerance=1e-6):
    E = M  # İlk tahmin
    iteration = 0
    while True:
        E_next = E - (E - e * math.sin(E) - M) / (1 - e * math.cos(E))
        if abs(E_next - E) < tolerance:
            break
        E = E_next
        iteration += 1
    return E


# Fonksiyon: Gerçek anomalinin ve diğer parametrelerin hesaplanması
def calculate_position(row):
    # Yörünge elemanlarını al
    M0 = math.radians(float(row[0]))        # Ortalama Anomali
    omega = math.radians(float(row[1]))  # Periapsis boylamı
    Omega = math.radians(float(row[2]))  # Yükselen düğüm boylamı
    I = math.radians(float(row[3]))  # Eğiklik
    e = float(row[4])         # Eksantriklik
    n = float(row[5])         # Günlük ortalama hareket (radyan/gün)
    a = float(row[6])         # Yarı büyük eksen (AU)
    epoch = datetime.strptime(row[7], "%Y%m%d")
    epoch_time = int(epoch.timestamp())

    # Gün farkını hesaplama
    difference_in_seconds = abs(entered_epoch_time - epoch_time)
    difference_in_days = difference_in_seconds // (24 * 3600)

    # Ortalama anomaliyi güncelleme ve mod 2*pi uygulama
    M = (M0 + math.radians(n * difference_in_days)) % (2 * math.pi)

    
    # Eksen anomali E hesaplanır
    E_rad = solve_kepler(M, e)

    # Gerçek anomali ν hesaplanır
    nu_rad = 2 * math.atan2(
        math.sqrt(1 + e) * math.sin(E_rad / 2),
        math.sqrt(1 - e) * math.cos(E_rad / 2)
    )

    # Yarıçap vektörü r hesaplanır
    r = a * (1 - e * math.cos(E_rad))

    # Perifokal koordinatlar (x', y')
    x_prime = r * math.cos(nu_rad)
    y_prime = r * math.sin(nu_rad)

    # Dönüşüm için gerekli trigonometrik değerler
    cos_Omega = math.cos(Omega)
    sin_Omega = math.sin(Omega)
    cos_I = math.cos(I)
    sin_I = math.sin(I)
    cos_omega = math.cos(omega)
    sin_omega = math.sin(omega)

    # Ekliptik koordinatlara dönüşüm
    x = (cos_Omega * cos_omega - sin_Omega * sin_omega * cos_I) * x_prime + \
        (-cos_Omega * sin_omega - sin_Omega * cos_omega * cos_I) * y_prime

    y = (sin_Omega * cos_omega + cos_Omega * sin_omega * cos_I) * x_prime + \
        (-sin_Omega * sin_omega + cos_Omega * cos_omega * cos_I) * y_prime

    z = (sin_omega * sin_I) * x_prime + (cos_omega * sin_I) * y_prime

    return M, x, y, z


with open(input_file, "r") as infile, open(output_file, "w", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Başlıkları yaz
    headers = next(reader)  # İlk satır başlıklar
    writer.writerow(["x", "y", "z"])

    # Her satır için hesaplamaları yap ve yaz
    for row in reader:
        M, x, y, z = calculate_position(row)
        writer.writerow([f"{x:.7f}", f"{y:.7f}", f"{z:.7f}"])


print(f"Konum verileri {output_file} dosyasına kaydedildi.")


Konum verileri 10k_20260101.csv dosyasına kaydedildi.


In [11]:
import math
from datetime import datetime

# Yörünge elemanları
a = 2.7666197  # Yarı büyük eksen (AU)
e = 0.0791840  # Eksantriklik
I = math.radians(10.58790)  # Eğiklik (radyan)
Omega = math.radians(80.25414)  # Çıkış düğümünün boylamı (radyan)
omega = math.radians(73.28579)  # Perihelion argümanı (radyan)
M0 = math.radians(145.84905)  # Epoch'taki ortalama anomali (radyan)
n = math.radians(0.21418047)  # Günlük ortalama hareket (radyan/gün)
epoch = datetime(2024, 11, 1)  # Epoch tarihi

# Güncel tarih
current_date = datetime.utcnow()

# 1. Ortalama Anomali (M) Hesaplama
days_since_epoch = (current_date - epoch).days
M = M0 + n * days_since_epoch

# 2. Kepler Denkleminin Çözülmesi (Eksen Anomali E)
def solve_kepler(M, e, tol=1e-6):
    E = M  # Başlangıç değeri
    while True:
        delta = E - e * math.sin(E) - M
        if abs(delta) < tol:
            break
        E -= delta / (1 - e * math.cos(E))
    return E

E = solve_kepler(M, e)

# 3. Gerçek Anomali (ν) Hesaplama
nu = 2 * math.atan2(
    math.sqrt(1 + e) * math.sin(E / 2),
    math.sqrt(1 - e) * math.cos(E / 2)
)

# 4. Yarıçap Vektörü (r) Hesaplama
r = a * (1 - e * math.cos(E))

# 5. Perifokal Koordinatlarda Konum (x', y')
x_prime = r * math.cos(nu)
y_prime = r * math.sin(nu)

# 6. Ekliptik Koordinatlara Dönüşüm
x = (
    (math.cos(Omega) * math.cos(omega) - math.sin(Omega) * math.sin(omega) * math.cos(I)) * x_prime
    + (-math.cos(Omega) * math.sin(omega) - math.sin(Omega) * math.cos(omega) * math.cos(I)) * y_prime
)
y = (
    (math.sin(Omega) * math.cos(omega) + math.cos(Omega) * math.sin(omega) * math.cos(I)) * x_prime
    + (-math.sin(Omega) * math.sin(omega) + math.cos(Omega) * math.cos(omega) * math.cos(I)) * y_prime
)
z = (
    (math.sin(omega) * math.sin(I)) * x_prime
    + (math.cos(omega) * math.sin(I)) * y_prime
)

# Güneş'e olan uzaklığı doğrulama
calculated_r = math.sqrt(x**2 + y**2 + z**2)

# Sonuçları yazdır
print(f"Ortalama Anomali (M): {math.degrees(M):.6f} derece")
print(f"Eksen Anomali (E): {math.degrees(E):.6f} derece")
print(f"Gerçek Anomali (ν): {math.degrees(nu):.6f} derece")
print(f"Yarıçap Vektörü (r): {r:.6f} AU")
print(f"Konum (x, y, z): ({x:.6f}, {y:.6f}, {z:.6f}) AU")
print(f"Doğrulanan Güneş'e Uzaklık (r): {calculated_r:.6f} AU")


Ortalama Anomali (M): 150.989381 derece
Eksen Anomali (E): 153.045859 derece
Gerçek Anomali (ν): 155.035052 derece
Yarıçap Vektörü (r): 2.961894 AU
Konum (x, y, z): (1.809734, -2.309212, -0.406474) AU
Doğrulanan Güneş'e Uzaklık (r): 2.961894 AU
